In [4]:
import dpkt
import socket

def mac_addr(address):
    """Convert a MAC address to a readable/printable string

       Args:
           address (str): a MAC address in hex form (e.g. '\x01\x02\x03\x04\x05\x06')
       Returns:
           str: Printable/readable MAC address
    """
    return ':'.join('%02x' % ord(b) for b in address)


def inet_to_str(inet):
    """Convert inet object to a string

        Args:
            inet (inet struct): inet network address
        Returns:
            str: Printable/readable IP address
    """
    # First try ipv4 and then ipv6
    try:
        return socket.inet_ntop(socket.AF_INET, inet)
    except ValueError:
        return socket.inet_ntop(socket.AF_INET6, inet)


In [5]:
# This is very optimistic. Won't work if the string is wrongly formatted
def bdecode(string):
    stack = []
    while (len(string)  > 0):
        if (string[0] == "d"):
            stack.append({})
            string = string[1:]
        elif (string[0] == "l"):
            stack.append([])
            string = string[1:]
        elif (string[0] == "i"):
            splitted = string[1:].split('e', 1)
            stack.append(int(splitted[0]))
            string = splitted[1]
        elif (string[0] == "e"):
            item = stack.pop()
            items = []
            while (len(stack) > 0 and item != {} and item != []):
                items.append(item)
                item = stack.pop()
            items.reverse()
            if (len(items) == 0):
                raise ValueError('empty list or dictionary')
            if (item == []):
                stack.append(items)
            elif (item == {}):
                dic = {}
                if (len(items) % 2 == 1):
                    raise ValueError('odd number of key/values in dictionary')
                while (len(items) != 0):
                    dic[items[0]] = items[1]
                    items = items[2:]
                stack.append(dic)
            string = string[1:]
                
        else:
            splitted = string.split(':', 1)
            str_len = int(splitted[0])
            string = splitted[1]
            stack.append(string[:str_len])
            string = string[str_len:]
        
    return stack[0]

In [16]:
f = open("pcaps/in_the_middle.pcap")
pcap = dpkt.pcap.Reader(f)

filters = [tracker_filter]
# For each packet in the pcap process the contents
for timestamp, buf in pcap:

    # Print out the timestamp in UTC
    # print 'Timestamp: ', str(datetime.datetime.utcfromtimestamp(timestamp))

    # Unpack the Ethernet frame (mac src/dst, ethertype)
    eth = dpkt.ethernet.Ethernet(buf)
    # print 'Ethernet Frame: ', mac_addr(eth.src), mac_addr(eth.dst), eth.type

    # Make sure the Ethernet frame contains an IP packet
    if not isinstance(eth.data, dpkt.ip.IP):
        # print 'Non IP Packet type not supported %s\n' % eth.data.__class__.__name__
        continue

    # Now unpack the data within the Ethernet frame (the IP packet)
    # Pulling out src, dst, length, fragment info, TTL, and Protocol
    ip = eth.data

    # Pull out fragment information (flags and offset all packed into off field, so use bitmasks)
    do_not_fragment = bool(ip.off & dpkt.ip.IP_DF)
    more_fragments = bool(ip.off & dpkt.ip.IP_MF)
    fragment_offset = ip.off & dpkt.ip.IP_OFFMASK
    pkt = ip.data

    for f in filters:
        is_torrent, output = f(pkt)
        if (is_torrent):
            protocol_str = "UNK"
            if isinstance(pkt, dpkt.tcp.TCP):
                protocol_str = "TCP"
            elif isinstance(pkt, dpkt.udp.UDP):
                protocol_str = "UDP"
            filter_name =  f.__name__
            sport, dport = "unk_port", "unk_port"
            try:
                sport = pkt.sport
                dport = pkt.dport
            except AttributeError:
                print "error"
        # Print out the info
            print 'Packet detected by %s' % filter_name
            print '%s: %s:%s -> %s:%s   (len=%d)' % \
              (protocol_str, inet_to_str(ip.src), sport, inet_to_str(ip.dst), dport, ip.len)
            print 'Output generated by filter: %s\n' % output

Packet detected by tracker_filter
UDP: 192.168.0.102:36360 -> 82.12.6.62:6881   (len=131)
Output generated by filter: {'a': {'id': 's\xd2K\xd6\xaeR\x90I\xf1\xf1\xbb\xe9\xeb\xb3\xa6\xdb<\x87\x0c\xe1', 'target': 's\xe0o\xfa\x00\x00lN\x00\x00\x1d\\\x00\x00\x1a\xa0\x00\x00"z'}, 'q': 'find_node', 'y': 'q', 't': '\x9fq\x00\x00', 'v': 'UT\xa6o'}

Packet detected by tracker_filter
UDP: 82.12.6.62:6881 -> 192.168.0.102:36360   (len=327)
Output generated by filter: {'y': 'r', 'ip': '\xd9\x81`h\x8e\x08', 'r': {'p': 36360, 'nodes': 's\xe7jl\xf2\x00\x85B{\xbe\x87\xf9\x7f\xa0\x01Y\xe2\x96g(M/@(\x1a\xe9s\xe4\xa1\x15\xb9g\xfdw\xce\x92\xc2\xf0\xaf<Z\xf0\xc0bjxm\x9b\x8c$\x1a\xe9s\xe1\xaa\x07$jZ\x93I\xff>\x080\xb4"6\xc10+\xbd%0GW\x1a\xe1s\xe9c\xeb\x83\xdf\xe1]\x88\xf6\xc5$\xa6!M~\x89\xa9\x19TU\xbf%9\xc8\xd5s\xec\xe6hk\xb2\xb4\xcf\x99\x9cE\x9de\x98\xbc\x8e%7\xa5H.\xf4\xee\x82\xaa\xdds\xed\x05\x9b6c@-D\x90\xabA\x07\x189\xb5\xc05\xacH.\xa6\xbf\x01\xdacs\xed\xfb}(\xad\xc1>\xcc\xbb-\xea\x01\x93\xa1\x8aL\x05\x

In [7]:
def tracker_filter(packet):
    data = str(packet.data)
    if (len(data) == 0 or data[0] not in ('d', 'l', 's')):
        return False, {}
    try:
        output = bdecode(data)
        return True, output
    except:
        return False, {}
    

SyntaxError: EOL while scanning string literal (<ipython-input-59-7ecbf8fd58bf>, line 1)